## Imports

In [ ]:
#DO NOT CHANGE THESE LINES
using Suppressor
@suppress begin
    using Pkg
    using DataFrames
    using LazyJSON
    using GLM 
    using MLJ 
    using MLJBase
    using CSV
    using Serialization
    using MLJScientificTypes
    using CategoricalArrays
    using StatsBase
    using StatsModels
end

## Paths

In [ ]:
# DO NOT CHANGE THESE LINES
ROOT_DIR = dirname(pwd())
MODEL_INPUTS_OUTPUTS = joinpath(ROOT_DIR, "model_inputs_outputs")
INPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = joinpath(INPUT_DIR, "schema")
DATA_DIR = joinpath(INPUT_DIR, "data")
TRAIN_DIR = joinpath(DATA_DIR, "training")
TEST_DIR = joinpath(DATA_DIR, "testing")
MODEL_PATH = joinpath(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = joinpath(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = joinpath(MODEL_ARTIFACTS_PATH, "ohe.jld2")
PREDICTOR_DIR_PATH = joinpath(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = joinpath(PREDICTOR_DIR_PATH, "predictor.jld2")
IMPUTATION_FILE = joinpath(MODEL_ARTIFACTS_PATH, "imputation.ser")
TOP_CATEGORIES = joinpath(MODEL_ARTIFACTS_PATH, "top_categories.ser")

if !isdir(MODEL_ARTIFACTS_PATH)
    mkdir(MODEL_ARTIFACTS_PATH)
end
if !isdir(PREDICTOR_DIR_PATH)
    mkdir(PREDICTOR_DIR_PATH)
end

### Reading the schema
The schema contains metadata about the datasets. We will use the scehma to get information about the type of each feature (NUMERIC or CATEGORICAL) and the id and target features, this will be helpful in preprocessing stage.

In [ ]:
# Reading a schema from a JSON file and extracting features
file_name = first(filter(x -> endswith(x, "json"), readdir(INPUT_SCHEMA_DIR)))
schema_path = joinpath(INPUT_SCHEMA_DIR, file_name)
schema_string = read(schema_path, String)  # Read file content as a string
schema = LazyJSON.parse(schema_string)  # Parse using LazyJSON

features = schema["features"]

# Identifying numeric, categorical, and nullable features
numeric_features = String[]
categorical_features = String[]
nullable_features = String[]

for f in features
    if f["dataType"] == "CATEGORICAL"
        push!(categorical_features, f["name"])
    else
        push!(numeric_features, f["name"])
    end
    if f["nullable"]
        push!(nullable_features, f["name"])
    end
end

# Extracting ID and target features
id_feature = schema["id"]["name"]
target_feature = schema["target"]["name"]


### Reading training data

In [ ]:
# Find the first CSV file in the TRAIN_DIR
file_name = first(filter(x -> endswith(x, ".csv"), readdir(TRAIN_DIR)))
file_path = joinpath(TRAIN_DIR, file_name)

# Read the CSV file into a DataFrame
df = CSV.File(file_path) |> DataFrame

# Display the first few rows of the DataFrame
first(df, 5)


## Data Preprocessing
Data preprocessing is very important before training the model, as the data may contain missing values in some cells. Moreover, most of the learning algorithms cannot work with categorical data, thus the data has to be encoded.

In this section we will impute the missing values and encode the categorical features. Afterwards the data will be ready to train the model.

##### Imputing missing data
> The median value will be used to impute missing values of the numeric features and the mode will be used to impute categorical features.

##### You can add your own preprocessing steps such as:
<ul>
<li>Normalization</li> <br>
<li>Outlier removal</li><br>
<li>Dropping or adding features</li><br>
</ul>

### Important note:
<p> 
Saving the values used for imputation during training step is crucial. These values will be used to impute missing data in the testing set. This is very important to avoid the well known problem of data leakage. During testing, you should not make any assumptions about the data in hand, alternatively anything needed during the testing phase should be learned from the training phase. This is why we are creating a dictionary of values used during training to reuse these values during testing.
</p>


In [ ]:
# Imputing missing data
imputation_values = Dict{String, Any}()

for column in nullable_features
    if column in numeric_features
        value = median(skipmissing(df[!, column]))
    else
        value = mode(skipmissing(df[!, column]))
    end
    df[!, column] = coalesce.(df[!, column], value)
    imputation_values[column] = value
end

# Serialize the imputation_values dictionary to a binary file
open(IMPUTATION_FILE, "w") do io
    serialize(io, imputation_values)
end


In [ ]:
# Saving the id and target columns in a different variable
ids = df[!, Symbol(id_feature)]
target = df[!, Symbol(target_feature)]

# Dropping the id and target from the DataFrame
select!(df, Not([Symbol(id_feature), Symbol(target_feature)]))

##### Encoding Categorical features
<p>
The id column is just an identifier for the training example, so we will exclude it during the encoding phase.<br>
Target feature will be label encoded in the next step.
</p>


In [ ]:
# Function to get top 10 categories
function get_top_categories(df, features, n=10)
    top_cats = Dict()
    for feature in features
        col_data = df[!, feature]
        category_counts = countmap(col_data)
        sorted_categories = sort(collect(category_counts), by=x->x[2], rev=true)
        
        # Take minimum between n and the number of unique categories
        num_categories = min(n, length(sorted_categories))
        
        top_cats[feature] = [x[1] for x in sorted_categories[1:num_categories]]
    end
    return top_cats
end

# Get top 10 categories for specific features
top_categories = get_top_categories(df, categorical_features)

# Function to one-hot encode only the top 10 categories
function one_hot_top_categories!(df, top_categories)
    for (feature, top_cats) in top_categories
        if length(top_cats) == 2  # Handle the binary case
            # Assuming the first category in top_cats is treated as 'true'
            new_col_name = "$(feature)_binary"
            df[!, new_col_name] = df[!, feature] .== top_cats[1]
        else  # Handle the general case
            for cat in top_cats
                new_col_name = "$(feature)_$(cat)"
                df[!, new_col_name] = df[!, feature] .== cat
            end
        end
        select!(df, Not(Symbol(feature)))  # Drop the original feature column
    end
end

# Apply one-hot encoding
one_hot_top_categories!(df, top_categories)

# Serialize the imputation_values dictionary to a binary file
open(TOP_CATEGORIES, "w") do io
    serialize(io, top_categories)
end


### Training the Regressor
We choose Linear Regression model, but feel free to try your own and compare the results.

In [ ]:
# Concatenate the new DataFrame with the existing 'df' DataFrame horizontally
df = hcat(df, DataFrame(target = target))

# Get all column names
all_columns = names(df)

# Remove the target variable to get only predictor variables
predictor_columns = filter(x -> x != "target", all_columns)

# Create Terms for predictors and response
predictor_terms = [Term(Symbol(col)) for col in predictor_columns]
response_term = Term(:target)

# Create the formula
formula_obj = FormulaTerm(response_term, sum(predictor_terms))

# Create a Linear Regression model and train it
model = lm(formula_obj, df)

# Your model is now trained on all predictor variables in 'df'

# BEGIN
    # If you want to perform additional operations on the model, you can do so here
    # For example, you can set hyperparameters, do cross-validation, etc.
    # model = ...
# END

# Saving the model to use it for predictions
open(PREDICTOR_FILE_PATH, "w") do io
    serialize(io, model)
end